In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm, trange
import firedrake
from firedrake import inner, grad, dx, ds, dS, jump, max_value, Constant
import irksome
from irksome import Dt

In [ ]:
num_levels = 5
mesh = firedrake.UnitDiskMesh(num_levels)
bernstein = firedrake.FiniteElement("Bernstein", "triangle", 2)
bernstein_dg = firedrake.BrokenElement(bernstein)
Q = firedrake.FunctionSpace(mesh, bernstein_dg)

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
u = firedrake.as_vector((-x[1], x[0]))

Create the initial data -- a spherical blip of radius 1/3 centered at the point (0.5, 0.0).

In [ ]:
ξ = Constant((0.5, 0.0))
r = Constant(1 / 3)
expr = max_value(0, 1 - inner(x - ξ, x - ξ) / r**2)
ϕ = firedrake.project(expr, Q)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(ϕ, axes=ax)
fig.colorbar(colors);

The sink term will be a spherical blob of radius 1/6 centered at the point (-0.5, 0).

In [ ]:
a_0 = Constant(10.0)
ζ = Constant((-0.5, 0.0))
ρ = Constant(1 / 6)
expr = -a_0 * max_value(0, 1 - inner(x - ζ, x - ζ) / ρ**2)
a = firedrake.project(expr, Q)

Create the variational form of the advection equation.
This step is the main difference between the present and the previous example.
Here we need to include the interface jump terms because the basis functions are discontinuous.

In [ ]:
ψ = firedrake.TestFunction(Q)
F_1 = (Dt(ϕ) * ψ - inner(ϕ * u, grad(ψ)) - a * ψ) * dx
ν = firedrake.FacetNormal(mesh)
f = ϕ * firedrake.max_value(0, inner(u, ν))
F_2 = jump(ϕ) * jump(ψ) * dS
F = F_1 + F_2

Create the bounds constraints and the solver; this is all the same as before.

In [ ]:
t = Constant(0.0)
num_steps = 512
dt = Constant(2 * π / num_steps)

lower = firedrake.Function(Q)
upper = firedrake.Function(Q)
upper.assign(+np.inf)
bounds = ("stage", lower, upper)

params = {
    "solver_parameters": {"snes_type": "vinewtonrsls"},
    "stage_type": "value",
    "basis_type": "Bernstein",
    "bounds": bounds,
}

method = irksome.RadauIIA(2)
solver = irksome.TimeStepper(F, method, t, dt, ϕ, **params)

Run the simulation.

In [ ]:
ϕs = [ϕ.copy(deepcopy=True)]

for step in trange(num_steps):
    solver.advance()
    t.assign(float(t) + float(dt))
    ϕs.append(ϕ.copy(deepcopy=True))

Make an animation of the solution.

In [ ]:
%%capture
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(
    ϕs[0], vmin=0.0, vmax=1.0, num_sample_points=4, axes=ax
)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(ϕ):
    colors.set_array(fn_plotter(ϕ))

In [ ]:
interval = 1e3 * 20 / num_steps
animation = FuncAnimation(fig, animate, frames=tqdm(ϕs), interval=interval)

In [ ]:
HTML(animation.to_html5_video())

Half-way through the simulation, the initial blob advects into the sink.
But by posing the problem as a variational inequality, we can make sure the solution does not go negative.
This is especially important in some physics problems where there are sinks that can remove mass from the system but where the key solution variable can't go negative.
Example 1: the solution variable represents the thickness of a glacier.
The ice can melt with sufficient solar heating, but the thickness of the glacier can't go negative.
Example 2: the solution variable represents the thickness of liquid water, e.g. from rainfall, flowing over a landscape.
This water can infiltrate into the subsurface aquifer, but the thickness of the surface water layer likewise can't go negative.

In [ ]:
volumes = [firedrake.assemble(ϕ * dx) for ϕ in ϕs]
fig, ax = plt.subplots()
ax.plot(volumes);

The DG scheme also has a small spurious mass gain, here only of the order of 0.3% instead of 0.9% for the CG scheme.

In [ ]:
(volumes[150] - volumes[0]) / volumes[0]